In [6]:
# Colab-ready Agent Task Validator

import json
from typing import Dict, List, Any, Tuple

# ---------------------------
# Embedded JSON Data
# ---------------------------

agent_tasks = [
    {
        "task_id": "T001",
        "user_query": "Find the weather in Tokyo and convert the temperature from Celsius to Fahrenheit",
        "expected_steps": [
            {
                "step_number": 1,
                "action": "get_weather",
                "parameters": {"city": "Tokyo", "unit": "celsius"},
                "reasoning": "User requested weather information for Tokyo"
            },
            {
                "step_number": 2,
                "action": "convert_temperature",
                "parameters": {"value": "${weather_temp}", "from_unit": "celsius", "to_unit": "fahrenheit"},
                "reasoning": "User requested temperature conversion from Celsius to Fahrenheit"
            }
        ]
    },
    {
        "task_id": "T003",
        "user_query": "Calculate 25% of 840 and save the result to a file",
        "expected_steps": [
            {
                "step_number": 1,
                "action": "calculate",
                "parameters": {"expression": "840 * 0.25"},
                "reasoning": "Calculate 25% of 840"
            },
            {
                "step_number": 2,
                "action": "save_to_file",
                "parameters": {"filename": "calculation_result.txt", "content": "${calculation_result}"},
                "reasoning": "Save the calculated result to a file"
            }
        ]
    }
]

agent_schemas = {
    "get_weather": {
        "description": "Retrieve weather information for a city",
        "required_params": ["city"],
        "optional_params": ["unit"],
        "param_types": {"city": "string", "unit": "string"}
    },
    "convert_temperature": {
        "description": "Convert temperature between units",
        "required_params": ["value", "from_unit", "to_unit"],
        "optional_params": [],
        "param_types": {"value": "number", "from_unit": "string", "to_unit": "string"}
    },
    "calculate": {
        "description": "Perform mathematical calculations",
        "required_params": ["expression"],
        "optional_params": [],
        "param_types": {"expression": "string"}
    },
    "save_to_file": {
        "description": "Save content to a file",
        "required_params": ["filename", "content"],
        "optional_params": [],
        "param_types": {"filename": "string", "content": "string"}
    }
}

# ---------------------------
# Helper Functions
# ---------------------------

def validate_param_type(value: Any, expected_type: str) -> Tuple[bool, str]:
    if expected_type == "string":
        if isinstance(value, str):
            return True, ""
        return False, f"Expected string, got {type(value).__name__}"
    elif expected_type == "integer":
        if isinstance(value, int):
            return True, ""
        return False, f"Expected integer, got {type(value).__name__}"
    elif expected_type == "number":
        if isinstance(value, (int, float)):
            return True, ""
        return False, f"Expected number, got {type(value).__name__}"
    else:
        return False, f"Unknown type: {expected_type}"

# ---------------------------
# Main Validator Class
# ---------------------------

class AgentTaskValidator:
    def __init__(self, tasks: List[Dict], schemas: Dict):
        self.tasks = tasks
        self.schemas = schemas

    def validate_step(self, step: Dict) -> Dict:
        action = step.get("action")
        params = step.get("parameters", {})
        result = {
            "step_number": step.get("step_number"),
            "action": action,
            "valid": True,
            "errors": []
        }
        if action not in self.schemas:
            result["valid"] = False
            result["errors"].append(f"Unknown action: {action}")
            return result
        schema = self.schemas[action]
        for req in schema["required_params"]:
            if req not in params:
                result["valid"] = False
                result["errors"].append(f"Missing required parameter: {req}")
        for param_name, param_value in params.items():
            if param_name in schema["param_types"]:
                is_valid, err = validate_param_type(param_value, schema["param_types"][param_name])
                if not is_valid:
                    result["valid"] = False
                    result["errors"].append(f"Param '{param_name}': {err}")
        return result

    def validate_task(self, task: Dict) -> Dict:
        task_result = {
            "task_id": task.get("task_id"),
            "user_query": task.get("user_query"),
            "steps_validation": [],
            "all_steps_valid": True
        }
        for step in task.get("expected_steps", []):
            step_result = self.validate_step(step)
            task_result["steps_validation"].append(step_result)
            if not step_result["valid"]:
                task_result["all_steps_valid"] = False
        return task_result

    def generate_report(self) -> Dict:
        report = {
            "summary": {
                "total_tasks": len(self.tasks),
                "valid_tasks": 0,
                "invalid_tasks": 0,
                "success_rate": "0%"
            },
            "tasks": []
        }
        valid_count = 0
        for task in self.tasks:
            task_result = self.validate_task(task)
            report["tasks"].append(task_result)
            if task_result["all_steps_valid"]:
                valid_count += 1
        report["summary"]["valid_tasks"] = valid_count
        report["summary"]["invalid_tasks"] = len(self.tasks) - valid_count
        report["summary"]["success_rate"] = f"{(valid_count/len(self.tasks)*100):.2f}%"
        return report

# ---------------------------
# Display Report
# ---------------------------

def display_report(report: Dict):
    print("="*80)
    print("AGENT TASK VALIDATION REPORT")
    print("="*80)

    summary = report["summary"]
    print(f"Total Tasks: {summary['total_tasks']}")
    print(f"Valid Tasks: {summary['valid_tasks']}")
    print(f"Invalid Tasks: {summary['invalid_tasks']}")
    print(f"Success Rate: {summary['success_rate']}")

    print("\nDetailed Task Results")
    print("="*80)
    for task in report["tasks"]:
        print(f"\nTask ID: {task['task_id']} | Query: {task['user_query']}")
        print(f"All Steps Valid: {'✓' if task['all_steps_valid'] else '✗'}")
        for step in task["steps_validation"]:
            print(f"  Step {step['step_number']} | Action: {step['action']} | Valid: {'✓' if step['valid'] else '✗'}")
            if step['errors']:
                for e in step['errors']:
                    print(f"    - Error: {e}")
        print("-"*80)

# ---------------------------
# Run Validator
# ---------------------------

validator = AgentTaskValidator(agent_tasks, agent_schemas)
report = validator.generate_report()
display_report(report)

# Save JSON report if needed
with open('agent_validation_report.json', 'w') as f:
    json.dump(report, f, indent=2)

print("\nReport saved as 'agent_validation_report.json'")


AGENT TASK VALIDATION REPORT
Total Tasks: 2
Valid Tasks: 1
Invalid Tasks: 1
Success Rate: 50.00%

Detailed Task Results

Task ID: T001 | Query: Find the weather in Tokyo and convert the temperature from Celsius to Fahrenheit
All Steps Valid: ✗
  Step 1 | Action: get_weather | Valid: ✓
  Step 2 | Action: convert_temperature | Valid: ✗
    - Error: Param 'value': Expected number, got str
--------------------------------------------------------------------------------

Task ID: T003 | Query: Calculate 25% of 840 and save the result to a file
All Steps Valid: ✓
  Step 1 | Action: calculate | Valid: ✓
  Step 2 | Action: save_to_file | Valid: ✓
--------------------------------------------------------------------------------

Report saved as 'agent_validation_report.json'
